# Imports


In [ ]:
!pip install pandas numpy matplotlib seaborn shap lime

pip install pandas numpy matplotlib seaborn shap lime


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

In [ ]:
# path_global = "/kaggle/input/fantasy-football/cleaned_merged_seasons.csv"
path_global = "./data/cleaned_merged_seasons.csv"

In [ ]:
df = pd.read_csv(path_global)
print(df.shape)
df.head()

In [ ]:
df_original = (
    df.copy()
)  # keep a copy of the original dataframe for visualizations at least

# EDA


In [ ]:
df.columns

In [ ]:
df.info()

Only null values in team_x -> will be handled in Data Cleaning


In [ ]:
duplicate_rows = df[df.duplicated(subset=["element", "kickoff_time"], keep=False)]


num_duplicates = len(duplicate_rows)
print(f"Number of rows with duplicate (element, kickoff_time): {num_duplicates}")

No duplicate entries


#### Minutes Analysis


In [ ]:
df[(df["position"] != "GK")]["minutes"].hist()

In [ ]:
df[(df["position"] != "GK") & (df["minutes"] > 0) & (df["minutes"] < 80)][
    "minutes"
].hist()

90 and 0 minutes are the most common minutes per match by multiple magnitudes.


#### Teams Check


In [ ]:
df["team_x"].unique()

In [ ]:
df["opp_team_name"].unique()

## Checking for numerical outliers


In [ ]:
# --- 1. Identify numerical columns automatically ---
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

print(f"Numeric columns found ({len(numeric_cols)}):")
print(numeric_cols)

# --- 2. Set visualization style ---
sns.set(style="whitegrid", font_scale=1.0)
plt.figure(figsize=(16, len(numeric_cols) * 0.6))

# --- 3. Draw boxplots for all numeric columns ---
# We'll melt the DataFrame so Seaborn can plot all columns in one figure
melted_df = df[numeric_cols].melt(var_name="Feature", value_name="Value")

sns.boxplot(
    data=melted_df,
    x="Value",
    y="Feature",
    orient="h",
    fliersize=3,  # smaller dots for outliers
    linewidth=0.7,
)

plt.title(
    "Boxplots of All Numerical Columns (Outlier Detection)", fontsize=14, weight="bold"
)
plt.xlabel("Value")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
counts = df.groupby(["season_x", "element"]).size()
less_than = 34
counts[counts <= less_than].hist(bins=less_than)

## Basic EDA


In [ ]:
# Set plot style
sns.set(style="whitegrid", palette="viridis", font_scale=1.1)
plt.rcParams["figure.figsize"] = (10, 5)


# ---- 1. BASIC OVERVIEW ----
def basic_overview(df):
    print("\n===== BASIC INFO =====")
    print(df.info())
    print("\n===== MISSING VALUES =====")
    print(df.isnull().sum().sort_values(ascending=False).head(10))
    print("\n===== DESCRIPTIVE STATS =====")
    display(df.describe(include="all").T)


# ---- 2. NUMERIC FEATURES ----
def numeric_analysis(df):
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
    print(f"\nNumeric features ({len(numeric_cols)}): {list(numeric_cols)}")

    # Histograms
    df[numeric_cols].hist(
        bins=30, figsize=(20, 15), color="steelblue", edgecolor="black"
    )
    plt.suptitle("Numeric Feature Distributions", fontsize=16)
    plt.show()

    # Boxplots for outliers
    for col in numeric_cols:
        plt.figure(figsize=(8, 3))
        sns.boxplot(x=df[col], color="lightgreen")
        plt.title(f"Boxplot: {col}")
        plt.show()

    # Correlation heatmap
    plt.figure(figsize=(12, 8))
    corr = df[numeric_cols].corr()
    sns.heatmap(corr, cmap="coolwarm", annot=False, linewidths=0.5)
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.show()


# ---- 3. CATEGORICAL FEATURES ----
def categorical_analysis(df):
    cat_cols = df.select_dtypes(include=["object", "bool"]).columns
    print(f"\nCategorical features ({len(cat_cols)}): {list(cat_cols)}")

    for col in cat_cols:
        plt.figure(figsize=(10, 4))
        df[col].value_counts(normalize=True).head(10).plot(
            kind="bar", color="cornflowerblue"
        )
        plt.title(f"Distribution of {col} (Top 10)")
        plt.ylabel("Frequency")
        plt.xlabel(col)
        plt.show()


# ---- 4. FEATURE RELATIONSHIPS ----
def feature_relationships(df):
    # Example: relationship between total_points and other key stats
    target = "total_points"
    if target in df.columns:
        num_cols = df.select_dtypes(include=["int64", "float64"]).columns
        top_corr = df[num_cols].corr()[target].sort_values(ascending=False)[1:11]
        print("\nTop correlations with total_points:")
        print(top_corr)

        plt.figure(figsize=(8, 5))
        sns.barplot(x=top_corr.values, y=top_corr.index, palette="viridis")
        plt.title("Top Features Correlated with Total Points")
        plt.show()

        for col in top_corr.index[:5]:
            plt.figure(figsize=(7, 4))
            sns.scatterplot(x=df[col], y=df[target], alpha=0.4)
            plt.title(f"{col} vs Total Points")
            plt.show()


# ---- 5. TIME / SEASON ANALYSIS ----
def time_analysis(df):
    if "kickoff_time" in df.columns:
        try:
            df["kickoff_time"] = pd.to_datetime(df["kickoff_time"], errors="coerce")
            df["year"] = df["kickoff_time"].dt.year
            df["month"] = df["kickoff_time"].dt.month

            plt.figure(figsize=(10, 4))
            df["year"].value_counts().sort_index().plot(
                kind="bar", color="lightseagreen"
            )
            plt.title("Games per Year")
            plt.xlabel("Year")
            plt.ylabel("Count")
            plt.show()

            plt.figure(figsize=(10, 4))
            df["month"].value_counts().sort_index().plot(kind="bar", color="orange")
            plt.title("Games per Month")
            plt.xlabel("Month")
            plt.ylabel("Count")
            plt.show()
        except Exception as e:
            print("Time analysis skipped:", e)


# ---- 6. FULL EDA PIPELINE ----
def full_eda(df):
    basic_overview(df)
    numeric_analysis(df)
    categorical_analysis(df)
    feature_relationships(df)
    time_analysis(df)
    print("\n✅ EDA Completed.")

In [ ]:
full_eda(df)

team_x does not have equal frequencies due to missing values


In [ ]:
df_trial = df[df.groupby(["season_x", "element"])["minutes"].transform("sum") > 0]

print("after_filter", len(df_trial), "b4", len(df))

In [ ]:
full_eda(df_trial)

No difference in the outliers of assists, bonus, bps, clean_sheets, creativity, goals_conceded, goals_scored, ict_index, influence, own goals, penalties_missed, penalties_saved, red_cards, saves, team_a_score, team_h_score, threat, total_points, value, yellow cards. <br>

Mostly all features contain mostly outliers. However these values are legitimate and not anomalies.


## Inconsistencies Check


In [ ]:
for team in df["team_x"].unique():
    if pd.isna(team):
        print("Was NaN pass")
        continue

    if team not in df["opp_team_name"].unique():
        print(f"Team {team} was in teams but not opp_teams")

All teams are consistently spelled


In [ ]:
df["position"].value_counts()

Goalkeepers are inconsistent (GKP & GK)


#### Checking for inconsistent total_points


##### Constants & Function definitions


In [ ]:
FPL_POINTS_MULTIPLIERS = {
    "GK": {
        "played_up_to_60_mins": 1,
        "played_60_plus_mins": 2,
        "goal_scored": 6,
        "assist": 3,
        "clean_sheet": 4,
        "penalty_save": 5,
        "every_3_saves": 1,
        "goals_conceded_per_2": -1,
        "yellow_card": -1,
        "red_card": -3,
        "own_goal": -2,
        "penalty_miss": -2,
        "defensive_contributions_2pts": 0,  # Does not apply for GK
    },
    "DEF": {
        "played_up_to_60_mins": 1,
        "played_60_plus_mins": 2,
        "goal_scored": 6,
        "assist": 3,
        "clean_sheet": 4,
        "penalty_save": 0,
        "every_3_saves": 0,
        "goals_conceded_per_2": -1,
        "yellow_card": -1,
        "red_card": -3,
        "own_goal": -2,
        "penalty_miss": -2,
        "defensive_contributions_2pts": 2,  # For 10+ clearances/blocks/interceptions/tackles
    },
    "MID": {
        "played_up_to_60_mins": 1,
        "played_60_plus_mins": 2,
        "goal_scored": 5,
        "assist": 3,
        "clean_sheet": 1,
        "penalty_save": 0,
        "every_3_saves": 0,
        "goals_conceded_per_2": 0,
        "yellow_card": -1,
        "red_card": -3,
        "own_goal": -2,
        "penalty_miss": -2,
        "defensive_contributions_2pts": 2,  # For 12+ clearances/blocks/interceptions/tackles/recoveries
    },
    "FWD": {
        "played_up_to_60_mins": 1,
        "played_60_plus_mins": 2,
        "goal_scored": 4,
        "assist": 3,
        "clean_sheet": 0,
        "penalty_save": 0,
        "every_3_saves": 0,
        "goals_conceded_per_2": 0,
        "yellow_card": -1,
        "red_card": -3,
        "own_goal": -2,
        "penalty_miss": -2,
        "defensive_contributions_2pts": 2,  # For 12+ clearances/blocks/interceptions/tackles/recoveries
    },
}

In [ ]:
def calculate_fpl_points(df, positions):
    df_temp = df.copy()
    for position in positions:
        mask = df["position"] == position
        df_temp.loc[mask, "fpl_points"] = (
            df_temp.loc[mask, "assists"] * FPL_POINTS_MULTIPLIERS[position]["assist"]
            + df_temp.loc[mask, "bonus"]
            + df_temp.loc[mask, "clean_sheets"]
            * FPL_POINTS_MULTIPLIERS[position]["clean_sheet"]
            + df_temp.loc[mask, "goals_conceded"]
            // 2
            * FPL_POINTS_MULTIPLIERS[position]["goals_conceded_per_2"]
            + df_temp.loc[mask, "goals_scored"]
            * FPL_POINTS_MULTIPLIERS[position]["goal_scored"]
            + (df_temp.loc[mask, "minutes"] >= 60) * 2
            + ((df_temp.loc[mask, "minutes"] > 0) & (df_temp.loc[mask, "minutes"] < 60))
            * 1
            + df_temp.loc[mask, "own_goals"]
            * FPL_POINTS_MULTIPLIERS[position]["own_goal"]
            + df_temp.loc[mask, "penalties_missed"]
            * FPL_POINTS_MULTIPLIERS[position]["penalty_miss"]
            + df_temp.loc[mask, "penalties_saved"]
            * FPL_POINTS_MULTIPLIERS[position]["penalty_save"]
            + df_temp.loc[mask, "red_cards"]
            * FPL_POINTS_MULTIPLIERS[position]["red_card"]
            + (df_temp.loc[mask, "saves"] // 3)
            * FPL_POINTS_MULTIPLIERS[position]["every_3_saves"]
            + df_temp.loc[mask, "yellow_cards"]
            * FPL_POINTS_MULTIPLIERS[position]["yellow_card"]
        )
    return df_temp

##### Applying function


In [ ]:
df_comparison_before = calculate_fpl_points(df, ["GK", "DEF", "MID", "FWD"])
df_comparison_before[
    df_comparison_before["fpl_points"] != df_comparison_before["total_points"]
]["name"].value_counts()

Example of a single match <br>
In which if is a defender then the goals conceded should result in a -1 but it does not, leaving him with 2 points (>=60 minutes played)<br>
Therefore this player was a MID on that season not a DEF


In [ ]:
df_comparison_before[
    (df_comparison_before["fpl_points"] != df_comparison_before["total_points"])
    & (df["name"] == "Eric Dier")
].iloc[0]

# Constants


In [ ]:
last_number_gws = 4

# Data Cleaning


#### Populating team_x for missing rows


In [ ]:
df_copy = df.copy()
print("init_shape", df_copy.shape)
# force keys to be strings to make sure datatypes are compatible when joining
for c in ["season_x", "GW", "fixture"]:
    df_copy[c] = df_copy[c].astype(str)


def teams_in_group(g):
    return set(g["opp_team_name"].dropna().astype(str).unique())


fixture_teams = (
    df_copy.groupby(["season_x", "GW", "fixture"])
    # FIX: Added include_groups=False to silence the DeprecationWarning
    .apply(teams_in_group, include_groups=False)
    .reset_index(name="teams")
)

# Keep only fixtures with exactly 2 distinct teams
fixture_teams = fixture_teams[fixture_teams["teams"].apply(lambda s: len(s) == 2)]

fixture_teams[["team_a", "team_b"]] = fixture_teams["teams"].apply(
    lambda s: pd.Series(sorted(list(s)))
)


df_merged = df_copy.merge(
    fixture_teams[["season_x", "GW", "fixture", "team_a", "team_b"]],
    on=["season_x", "GW", "fixture"],
    how="left",
)

# mask to manage only rows who have team_x as null
mask_null_teamx = df_merged["team_x"].isna() & df_merged["team_a"].notna()

# FIX: Initialize the column with a string-compatible dtype ('object')
# to silence the FutureWarning when assigning team names (strings) later.
df_merged["team_x_inferred"] = np.nan
df_merged["team_x_inferred"] = df_merged["team_x_inferred"].astype("object")


# where opponent == team_a -> set to team_b
mask_a = mask_null_teamx & (
    df_merged["opp_team_name"].astype(str) == df_merged["team_a"]
)
df_merged.loc[mask_a, "team_x_inferred"] = df_merged.loc[mask_a, "team_b"]

# where opponent == team_b -> set to team_a
mask_b = mask_null_teamx & (
    df_merged["opp_team_name"].astype(str) == df_merged["team_b"]
)
df_merged.loc[mask_b, "team_x_inferred"] = df_merged.loc[mask_b, "team_a"]

df_merged["team_x_filled"] = df_merged["team_x"].fillna(df_merged["team_x_inferred"])

df_result = df.copy()
# replace nulls in original team_x with filled values
df_result["team_x"] = df_result["team_x"].fillna(df_merged["team_x_filled"])
print(df_result.shape)
df_result.head()

#### Unify Goalkeeper representation


In [ ]:
df["position"] = df["position"].replace({"GKP": "GK"})

Converted GKP to GK


#### Handle Minimum Number of GWS


In [ ]:
print("before_filter", len(df))
minimum_gws = 2  # Can change according to performance
df = df[df.groupby(["season_x", "element"])["element"].transform("size") >= minimum_gws]
print("after_filter", len(df))

Removed players that had less than `minimum_gws` gameweeks in a single season


#### Fixing players with incorrect FPL point (wrong positions)


##### Constant and Function Definitions


In [ ]:
# Define all position updates in one place
position_updates = [
    {
        "names": ["Jeffrey Schlupp", "Jairo Riedewald", "Declan Rice"],
        "seasons": ["2016-17", "2017-18"],
        "position": "DEF",
    },
    {
        "names": ["James Milner"],
        "seasons": ["2017-18"],
        "position": "DEF",
    },
    {
        "names": [
            "Eric Dier",
            "Fernando Luiz Rosa",
            "Ashley Young",
            "Daniel Amartey",
            "Robert Kenedy Nunes do Nascimento",
            "Ainsley Maitland-Niles",
            "Aaron Wan-Bissaka",
            "Oleksandr Zinchenko",
            "Anthony Martial",
            "Michail Antonio",
            "Richarlison de Andrade",
        ],
        "seasons": ["2016-17", "2017-18"],
        "position": "MID",
    },
    {
        "names": ["Roberto Firmino", "Jay Rodriguez", "Joshua King"],
        "seasons": ["2016-17"],
        "position": "MID",
    },
    {
        "names": ["Marcus Rashford", "Ayoze Pérez", "Pierre-Emerick Aubameyang"],
        "seasons": ["2016-17", "2017-18"],
        "position": "FWD",
    },
]


In [ ]:
def apply_position_updates(df, updates):
    df_temp = df.copy()
    for update in updates:
        condition = df["name"].isin(update["names"]) & df["season_x"].isin(
            update["seasons"]
        )
        df_temp.loc[condition, "position"] = update["position"]
    return df_temp

##### Applying function


In [ ]:
df_positions = apply_position_updates(df, position_updates)

In [ ]:
df_comparison_after = calculate_fpl_points(df_positions, ["GK", "DEF", "MID", "FWD"])
df_comparison_after[
    df_comparison_after["fpl_points"] != df_comparison_after["total_points"]
]["name"].value_counts()

In [ ]:
df = df_positions.copy()
print(df.shape)
df.head()

After corrections in position -> Correct FPL points calculated


# Data Engineering Questions


In [ ]:
from IPython.display import display, Markdown
import matplotlib as mpl

In [ ]:
rename_map = {
    "season_x": "season",
    "team_x": "team",
    "round": "gameweek",
    "name": "player_name",
}
df_original.rename(
    columns={k: v for k, v in rename_map.items() if k in df_original.columns},
    inplace=True,
)

# ---------- PREPROCESS ----------
for col in ["season", "position", "player_name", "team"]:
    if col in df_original.columns:
        df_original[col] = df_original[col].astype(str).str.strip()

for c in ["gameweek", "total_points"]:
    if c in df_original.columns:
        df_original[c] = pd.to_numeric(df_original[c], errors="coerce")

# Compute `form` feature (rolling avg over past 4 GWs ÷ 10)
df_original.sort_values(["season", "player_name", "gameweek"], inplace=True)
df_original["form"] = df_original.groupby(["season", "player_name"])[
    "total_points"
].transform(lambda s: s.rolling(window=4, min_periods=1).mean() / 10.0)

In [ ]:
def show_position_insights(season_pos_avg_df, title="Insights"):
    # Focus on the four main positions
    dfp = season_pos_avg_df[
        season_pos_avg_df["position"].isin(["GK", "DEF", "MID", "FWD"])
    ].copy()
    if dfp.empty:
        display(Markdown(f"### 🔎 {title}\nNo data available for GK/DEF/MID/FWD."))
        return

    # Overall averages across seasons (who scores most on average?)
    overall = (
        dfp.groupby("position")["avg_points_per_player"]
        .mean()
        .sort_values(ascending=False)
    )

    # Rank positions within each season (1 = best)
    ranks = dfp.assign(
        rank=dfp.groupby("season")["avg_points_per_player"].rank(
            ascending=False, method="min"
        )
    )
    top_by_season = (
        ranks[ranks["rank"] == 1]
        .groupby("position")
        .size()
        .sort_values(ascending=False)
    )

    # Best / worst season for the overall top position
    best_line = worst_line = "-"
    try:
        top_pos = overall.index[0]
        top_pos_rows = dfp[dfp["position"] == top_pos]
        best_row = top_pos_rows.sort_values(
            "avg_points_per_player", ascending=False
        ).iloc[0]
        worst_row = top_pos_rows.sort_values(
            "avg_points_per_player", ascending=True
        ).iloc[0]
        best_line = f"**Best season for {top_pos}:** {best_row['season']} — {best_row['avg_points_per_player']:.1f}"
        worst_line = f"**Lowest season for {top_pos}:** {worst_row['season']} — {worst_row['avg_points_per_player']:.1f}"
    except Exception:
        pass

    # Compose Markdown
    lines = [
        f"### 🔎 {title}",
        f"- **Overall order (avg points per player):** "
        + ", ".join([f"{p} ({overall[p]:.1f})" for p in overall.index]),
        f"- **Most often #1:** {top_by_season.index[0]} in {int(top_by_season.iloc[0])} of {dfp['season'].nunique()} seasons."
        if len(top_by_season)
        else "- **Most often #1:** N/A",
        f"- {best_line}",
        f"- {worst_line}",
    ]
    display(Markdown("\n".join(lines)))

In [ ]:
display(
    Markdown("## 🧮 Q(a): Which positions score the most on average — **per season**?")
)

display(Markdown("## No filters (everyone included)"))

# 0) Normalize column types
for c in ["total_points", "minutes"]:
    if c in df_original.columns:
        df_original[c] = pd.to_numeric(df_original[c], errors="coerce")

# 1) Normalize position labels so we don't lose GKP etc.
pos_map = {
    "GKP": "GK",
    "GK": "GK",
    "DEF": "DEF",
    "DF": "DEF",
    "MID": "MID",
    "MF": "MID",
    "FWD": "FWD",
    "FW": "FWD",
    "ST": "FWD",
}
df_original["position"] = (
    df_original["position"]
    .astype(str)
    .str.strip()
    .str.upper()
    .map(pos_map)
    .fillna("OTHER")
)

# 2) Aggregate to player-season totals (sum per GW → season total)
player_season = (
    df_original.groupby(["season", "position", "player_name"], dropna=False)[
        "total_points"
    ]
    .sum()
    .reset_index(name="player_total_points")
)

# 3) Average across players for each (season, position)
season_pos_avg = (
    player_season.groupby(["season", "position"], dropna=False)["player_total_points"]
    .mean()
    .reset_index(name="avg_points_per_player")
)

# 4) Plot — one chart per season (positions GK/DEF/MID/FWD shown if present)
POS_COLORS = {"GK": "#4C78A8", "DEF": "#F58518", "MID": "#54A24B", "FWD": "#E45756"}
pos_order = ["GK", "DEF", "MID", "FWD"]

mpl.rcParams.update(
    {
        "figure.dpi": 120,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.titleweight": "bold",
        "axes.grid": True,
        "grid.alpha": 0.25,
        "grid.linestyle": "--",
        "grid.linewidth": 0.6,
    }
)

seasons = sorted(season_pos_avg["season"].astype(str).unique())
for s in seasons:
    sdf = season_pos_avg[season_pos_avg["season"].astype(str) == s].copy()

    # Keep the 4 main positions if they exist; ignore "OTHER" in visuals
    sdf = sdf[sdf["position"].isin(pos_order)]
    sdf["position"] = pd.Categorical(
        sdf["position"], categories=pos_order, ordered=True
    )
    sdf = sdf.sort_values("position")

    positions = sdf["position"].tolist()
    values = sdf["avg_points_per_player"].tolist()
    colors = [POS_COLORS.get(p, "#737373") for p in positions]

    fig, ax = plt.subplots(figsize=(8.5, 5.2))
    bars = ax.bar(
        range(len(positions)), values, edgecolor="black", linewidth=0.6, alpha=0.95
    )
    for b, c in zip(bars, colors):
        b.set_facecolor(c)

    ymax = max(values) if values else 0
    ax.set_ylim(0, ymax * 1.15 if ymax else 1)
    ax.set_xticks(range(len(positions)))
    ax.set_xticklabels(positions)
    ax.set_ylabel("Average points per player (season total)")
    ax.set_title(f"Season {s} — Avg Points per Player by Position (no filters)")

    for i, v in enumerate(values):
        ax.text(
            i,
            v * 1.01 if ymax else 0.02,
            f"{v:.1f}",
            ha="center",
            va="bottom",
            fontsize=10,
            fontweight="bold",
        )

    fig.tight_layout()
    plt.show()
display(
    Markdown(
        "✅ **Note:** Each chart shows, for that season, the mean of each player's **season-total** points within the position. "
        "Equivalently: (sum of all players’ season-total points in the position) ÷ (number of players in the position)."
    )
)
show_position_insights(season_pos_avg, "Insights — (No filters)")

In [ ]:
display(
    Markdown("## 🧮 Q(a): Which positions score the most on average — **per season**?")
)


display(Markdown("## With filters (active players only, minutes > 0)"))

MIN_MINUTES = 0

# 0) Normalize column types
for c in ["total_points", "minutes", "gameweek"]:
    if c in df_original.columns:
        df_original[c] = pd.to_numeric(df_original[c], errors="coerce")

# 1) Normalize position labels
pos_map = {
    "GKP": "GK",
    "GK": "GK",
    "DEF": "DEF",
    "DF": "DEF",
    "MID": "MID",
    "MF": "MID",
    "FWD": "FWD",
    "FW": "FWD",
    "ST": "FWD",
}
df_original["position"] = (
    df_original["position"]
    .astype(str)
    .str.strip()
    .str.upper()
    .map(pos_map)
    .fillna("OTHER")
)

# 2) Aggregate to player-season totals (sum per GW → season total)
player_season = (
    df_original.groupby(["season", "position", "player_name"], dropna=False)
    .agg(
        player_total_points=("total_points", "sum"),
        player_total_minutes=("minutes", "sum"),
    )
    .reset_index()
)

# 3) FILTER: keep only players who actually played enough
player_season_f = player_season[
    player_season["player_total_minutes"].fillna(0) > MIN_MINUTES
]

# 4) Average across players for each (season, position)
season_pos_avg_f = (
    player_season_f.groupby(["season", "position"], dropna=False)["player_total_points"]
    .mean()
    .reset_index(name="avg_points_per_player")
)

# 5) Plot — one chart per season
POS_COLORS = {"GK": "#4C78A8", "DEF": "#F58518", "MID": "#54A24B", "FWD": "#E45756"}
pos_order = ["GK", "DEF", "MID", "FWD"]

mpl.rcParams.update(
    {
        "figure.dpi": 120,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.titleweight": "bold",
        "axes.grid": True,
        "grid.alpha": 0.25,
        "grid.linestyle": "--",
        "grid.linewidth": 0.6,
    }
)

seasons = sorted(season_pos_avg_f["season"].astype(str).unique())
for s in seasons:
    sdf = season_pos_avg_f[season_pos_avg_f["season"].astype(str) == s].copy()

    # Keep only the 4 main positions
    sdf = sdf[sdf["position"].isin(pos_order)]
    sdf["position"] = pd.Categorical(
        sdf["position"], categories=pos_order, ordered=True
    )
    sdf = sdf.sort_values("position")

    positions = sdf["position"].tolist()
    values = sdf["avg_points_per_player"].tolist()
    colors = [POS_COLORS.get(p, "#737373") for p in positions]

    fig, ax = plt.subplots(figsize=(8.5, 5.2))
    bars = ax.bar(
        range(len(positions)), values, edgecolor="black", linewidth=0.6, alpha=0.95
    )
    for b, c in zip(bars, colors):
        b.set_facecolor(c)

    ymax = max(values) if values else 0
    ax.set_ylim(0, ymax * 1.15 if ymax else 1)
    ax.set_xticks(range(len(positions)))
    ax.set_xticklabels(positions)
    ax.set_ylabel("Average points per player (season total)")
    ax.set_title(
        f"Season {s} — Avg Points per Player by Position (minutes > {MIN_MINUTES})"
    )

    for i, v in enumerate(values):
        ax.text(
            i,
            v * 1.01 if ymax else 0.02,
            f"{v:.1f}",
            ha="center",
            va="bottom",
            fontsize=10,
            fontweight="bold",
        )

    fig.tight_layout()
    plt.show()
display(
    Markdown(
        "✅ **Note:** Each chart shows, for that season, the mean of each player's **season-total** points within the position. "
        "Equivalently: (sum of all players’ season-total points in the position) ÷ (number of players in the position)."
    )
)
show_position_insights(season_pos_avg_f, f"Insights — (minutes > {MIN_MINUTES})")

In [ ]:
# GW if present
if "GW" in df_original.columns and "gameweek" not in df_original.columns:
    df_original.rename(columns={"GW": "gameweek"}, inplace=True)

# make numerics truly numeric and safe for rolling averages
df_original["total_points"] = pd.to_numeric(
    df_original.get("total_points"), errors="coerce"
).fillna(0)
df_original["gameweek"] = pd.to_numeric(df_original.get("gameweek"), errors="coerce")

# drop dupes (helps both features and plots)
df_original = df_original.drop_duplicates()

df_original = df_original.sort_values(["season", "player_name", "gameweek"])

# --- Recompute 'form' in case NaN handling changed ---
df_original["form"] = df_original.groupby(["season", "player_name"])[
    "total_points"
].transform(lambda s: s.rolling(window=4, min_periods=1).mean() / 10.0)

# pick the 2022–23 season robustly (fallback to latest if not found)
season_2223 = next(
    (s for s in df_original["season"].astype(str).unique() if "22" in s and "23" in s),
    None,
)
if season_2223 is None:
    season_2223 = sorted(df_original["season"].astype(str).unique())[-1]

df_original_2223 = df_original[df_original["season"].astype(str) == season_2223].copy()

# compute the two top-5 lists used by your small multiples ---
# Top-5 by season total points
season_totals = (
    df_original_2223.groupby("player_name")["total_points"]
    .sum()
    .sort_values(ascending=False)
)
top5_total = season_totals.head(5).index.tolist()

# Top-5 by average form
avg_form = (
    df_original_2223.groupby("player_name")["form"].mean().sort_values(ascending=False)
)
top5_form = avg_form.head(5).index.tolist()

In [ ]:
# =======================================================
# Q(b): Top-5 by Total Points — Form Evolution (One Graph)
# =======================================================
display(Markdown("## ⚽ Q(b): Form Evolution of Top-5 Players (2022–23)"))

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

fig, ax = plt.subplots(figsize=(11, 6))
palette = sns.color_palette("Set2", len(top5_total))

for color, p in zip(palette, top5_total):
    sub = (
        df_original_2223[df_original_2223["player_name"] == p]
        .sort_values("gameweek")[["gameweek", "form"]]
        .drop_duplicates(subset="gameweek")
    )

    # line + markers for each player’s form
    ax.plot(
        sub["gameweek"],
        sub["form"],
        label=p,
        linewidth=2.4,
        color=color,
        alpha=0.9,
        marker="o",
        markersize=5,
    )

# --- formatting ---
ax.set_title(
    f"Form Evolution — Top-5 Players by Total Points ({season_2223})",
    fontsize=15,
    fontweight="bold",
    pad=12,
)
ax.set_xlabel("Gameweek", fontsize=12)
ax.set_ylabel("Form (rolling 4 GW avg ÷ 10)", fontsize=12)
ax.set_xlim(0, 38)
ax.set_ylim(0, df_original_2223["form"].max() + 0.2)
ax.legend(title="Player", frameon=False, fontsize=10)
ax.grid(True, alpha=0.3)
sns.despine()
plt.tight_layout()
plt.show()

# ============================
# Insights: Top Points vs Best Form
# ============================

from IPython.display import Markdown, display

# --- compute season totals per player (use max since total_points is cumulative by GW) ---
season_totals = (
    df_original_2223.drop_duplicates(subset=["player_name", "gameweek"])
    .groupby("player_name")["total_points"]
    .max()
)

# --- compute season-average form per player ---
avg_form = (
    df_original_2223.drop_duplicates(subset=["player_name", "gameweek"])
    .groupby("player_name")["form"]
    .mean()
)

# --- info for players already in your top5_total list ---
top5_total_series = season_totals.reindex(top5_total).sort_values(ascending=False)

# --- top-5 by average form across the season ---
top5_form_series = avg_form.sort_values(ascending=False).head(5)

# --- sets for explicit comparison ---
set_total = set(top5_total_series.index)
set_form = set(top5_form_series.index)

overlap = list(set_total & set_form)
only_total = list(set_total - set_form)
only_form = list(set_form - set_total)


# markdown lists
def md_list_from_series(title, s, unit=""):
    lines = [f"**{title}**"]
    for i, (name, val) in enumerate(s.items(), start=1):
        lines.append(f"{i}. **{name}** — {val:.2f}{unit}")
    return "\n".join(lines)


insights_md = f"""
## 🧠 Insights — Top Points vs. Best Form

{md_list_from_series("Top-5 by Total Points (Season Total)", top5_total_series, unit=" pts")}

{md_list_from_series("Top-5 by Best Average Form (Season Avg.)", top5_form_series, unit=" form")}

**Comparison**
- **Overlap (appear in both lists):** {", ".join(overlap) if overlap else "—"}
- **Only in Top-5 by Total Points:** {", ".join(only_total) if only_total else "—"}
- **Only in Top-5 by Best Average Form:** {", ".join(only_form) if only_form else "—"}

**Takeaway:** Players in both lists combined strong **short-term performance (form)** with sustained output, which typically translates into high **season totals**. Those appearing **only in form** may have had excellent bursts without matching the full-season accumulation; those **only in totals** likely accumulated consistently even if their average form wasn’t among the very top.
"""

display(
    Markdown(f"""
### 🔎 Interpretation 
- The plot shows how the **form (short-term average performance)** of the top five players by total season points changed across the 38 gameweeks of the 2022–23 FPL season.
- Each line represents one player's form, calculated as the average of their last four gameweeks' total points divided by 10.
- **Answer:** The top players **in form** are mostly the same as those with the highest total points — especially Haaland, Kane, Salah, and Ødegaard — indicating that consistent high form correlates strongly with high season totals.
""")
)

display(Markdown(insights_md))

In [ ]:
# # ======================= old =========================================
# # =======================================================
# # Q(a): Average total points by position across seasons
# # =======================================================
# display(Markdown("## 🧮 Q(a): Which positions score the most on average?"))

# pos_season_sum = (
#     df.groupby(["season", "position"])["total_points"]
#       .sum()
#       .reset_index()
# )

# pos_avg = (
#     pos_season_sum.groupby("position")["total_points"]
#       .mean()
#       .sort_values(ascending=False)
#       .reset_index()
#       .rename(columns={"total_points": "avg_total_points"})
# )

# # --- Visualization: bars with labels inside ---
# fig, ax = plt.subplots(figsize=(8, 5))
# bar = sns.barplot(
#     x="position", y="avg_total_points",
#     data=pos_avg, palette="magma", ax=ax
# )


# ymax = pos_avg["avg_total_points"].max()
# for i, row in pos_avg.iterrows():
#     val = row["avg_total_points"]

#     color = "white" if val > 0.35 * ymax else "black"
#     ax.text(
#         i, val * 0.55, f"{int(val):,}",
#         ha="center", va="center", fontsize=10, fontweight="bold", color=color
#     )

# ax.set_title("Average Total Points by Position (Across Seasons)", fontsize=14, fontweight="bold")
# ax.set_xlabel("Player Position", fontsize=12)
# ax.set_ylabel("Average Total Points (per season)", fontsize=12)
# plt.tight_layout()
# plt.show()

# display(Markdown("""
# ✅ **Insight:**
# Midfielders and forwards generally contribute the highest total points on average,
# while defenders and goalkeepers score lower due to fewer goal/assist opportunities.
# """))

In [ ]:
# # =======================================================
# # Top-5 by Season Total Points (2022–23)
# # Each subplot shows total points evolution
# # =======================================================
# display(Markdown("## ⚽ Q(b.1): Top-5 by Season Total Points (2022–23)"))

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(style="whitegrid")

# players = top5_total
# n = len(players)

# fig, axes = plt.subplots(n, 1, figsize=(11, 2.4*n), sharex=True, sharey=True)

# for ax, p in zip(axes, players):
#     sub = (
#         df_2223[df_2223["player_name"] == p]
#         .sort_values("gameweek")[["gameweek", "total_points"]]
#         .drop_duplicates(subset="gameweek")
#     )
#     # line and scatter for total_points
#     ax.plot(sub["gameweek"], sub["total_points"], color="#2ca02c", linewidth=2.5)
#     ax.scatter(sub["gameweek"], sub["total_points"], color="#2ca02c", s=20, alpha=0.7)

#     ax.set_ylabel("Total Points", fontsize=10)
#     ax.set_title(p, loc="left", fontsize=12, fontweight="bold")
#     ax.grid(True, alpha=0.3)

# axes[-1].set_xlabel("Gameweek", fontsize=11)
# axes[0].set_ylim(0, df_2223["total_points"].max() + 2)
# fig.suptitle(f"Total Points by Gameweek — Top-5 Players ({season_2223})",
#              fontsize=14, fontweight="bold", y=0.99)
# fig.tight_layout(rect=[0, 0, 1, 0.98])
# plt.show()

In [ ]:
# # =======================================================
# # Top-5 by Average Form (2022–23)
# # =======================================================
# display(Markdown("## ⚽ Q(b.2): Top-5 by Average Form (2022–23)"))

# sns.set(style="whitegrid")

# players = top5_form
# n = len(players)

# fig, axes = plt.subplots(n, 1, figsize=(11, 2.4*n), sharex=True, sharey=True)

# for ax, p in zip(axes, players):
#     sub = (
#         df_2223[df_2223["player_name"] == p]
#         .sort_values("gameweek")[["gameweek", "form"]]
#         .drop_duplicates(subset="gameweek")
#     )
#     ax.plot(sub["gameweek"], sub["form"], color="#d62728", linewidth=2.5)
#     ax.scatter(sub["gameweek"], sub["form"], color="#d62728", s=18, alpha=0.7)

#     ax.set_ylabel("Form", fontsize=10)
#     ax.set_title(p, loc="left", fontsize=12, fontweight="bold")
#     ax.grid(True, alpha=0.3)

# axes[-1].set_xlabel("Gameweek", fontsize=11)
# fig.suptitle(f"Form by Gameweek — Top-5 by Average Form ({season_2223})",
#              fontsize=14, fontweight="bold", y=0.99)
# fig.tight_layout(rect=[0, 0, 1, 0.98])
# plt.show()
# # --- Inline summary ---
# overlap = sorted(set(top5_total) & set(top5_form))
# display(Markdown(f"""
# ✅ **Top-5 by Total Points:** {', '.join(top5_total)}
# ✅ **Top-5 by Average Form:** {', '.join(top5_form)}
# 🔁 **Overlap:** {', '.join(overlap) if overlap else 'None'}

# **Interpretation:**
# The overlap shows players like *{', '.join(overlap)}* maintained both high total points
# and consistent form. Others (like short-term performers) peaked briefly but didn’t sustain
# their performance across the season.
# """))

zeina's part


# Feature Engineering


#### Adding upcoming_total_points column (y predict)


In [ ]:
df.sort_values(by=["season_x", "element", "kickoff_time"], inplace=True)
df["upcoming_total_points"] = df.groupby(["season_x", "element"])["total_points"].shift(
    -1
)
mask_gw_38 = df["GW"] == 38
df.loc[mask_gw_38, "upcoming_total_points"] = df.loc[mask_gw_38, "total_points"]
print(df.shape)
df.head()

#### Adding team_score & opponent_score columns


In [ ]:
def add_team_scores(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df["team_score"] = np.where(df["was_home"], df["team_h_score"], df["team_a_score"])
    df["opponent_score"] = np.where(
        df["was_home"], df["team_a_score"], df["team_h_score"]
    )

    return df


df = add_team_scores(df)
print(df.shape)
df.head()

Needed to infer wins, losses, and draws


#### Adding form column


In [ ]:
form_df = df.copy()

form_df = form_df.sort_values(
    by=["season_x", "element", "GW", "kickoff_time"]
).reset_index(drop=True)


def calculate_form(group):
    # Aggregate per GW (combine DGWs)
    gw_points = group.groupby("GW", as_index=False)["total_points"].sum()

    # Compute rolling sum of last 4 GWs (excluding current GW)
    gw_points["form"] = (
        gw_points["total_points"]
        # .shift(1)  # exclude current GW
        .rolling(window=4, min_periods=1)  # use up to 4 previous GWs
        .sum()
        .fillna(0)
        / 10  # scale like FPL (divided by 10)
    )

    # Merge back so both DGW rows share same form
    return group.merge(gw_points[["GW", "form"]], on="GW", how="left")


# Apply to each player (element) per season
form_df = form_df.groupby(["season_x", "element"], group_keys=False).apply(
    calculate_form
)

# Optional: fill NaN (shouldn’t appear except for first GW)
form_df["form"] = form_df["form"].fillna(0)

# Preview result
print(form_df.shape)
form_df.head()

In [ ]:
df = form_df.copy()
df.head()

#### Creating teams_df


A df which stores performance stats of teams


In [ ]:
df.shape

In [ ]:
# --- 2. Feature Engineering ---

# Step A: Isolate team-match data and remove duplicates, include kickoff_time
team_match_data = (
    df[["season_x", "team_x", "GW", "kickoff_time", "team_score", "opponent_score"]]
    .drop_duplicates()
    .copy()
)

# Step B: Sort values to ensure correct cumulative/rolling calculation order
team_match_data = team_match_data.sort_values(
    by=["season_x", "team_x", "kickoff_time", "GW"]
)

# Step C: Determine win, loss, or draw for each individual match
conditions = [
    team_match_data["team_score"] > team_match_data["opponent_score"],
    team_match_data["team_score"] < team_match_data["opponent_score"],
    team_match_data["team_score"] == team_match_data["opponent_score"],
]
team_match_data["win"] = np.select(conditions, [1, 0, 0], default=0)
team_match_data["loss"] = np.select(conditions, [0, 1, 0], default=0)
team_match_data["draw"] = np.select(conditions, [0, 0, 1], default=0)

# --- Create the final DataFrame, starting with the base data ---
teams_df = team_match_data.copy()

# Step D: Calculate Cumulative Stats
cols_to_sum = ["team_score", "opponent_score", "win", "loss", "draw"]
cumulative_cols = [f"cumulative_{col}" for col in cols_to_sum]
teams_df[cumulative_cols] = teams_df.groupby(["season_x", "team_x"])[
    cols_to_sum
].cumsum()

# --- NEW STEP D.5: Identify which teams are active in each season ---
active_teams_index = (
    df.dropna(subset=["team_score", "opponent_score"])
    .set_index(["season_x", "team_x"])
    .index.unique()
)
teams_df["is_active"] = teams_df.set_index(["season_x", "team_x"]).index.isin(
    active_teams_index
)

# --- MODIFIED STEP E: Calculate Ranks for Active Teams Only ---
rank_cols = []

# Rank "good" stats: higher is better (wins, goals scored)
for col in ["cumulative_team_score", "cumulative_win"]:
    rank_col_name = f"{col}_rank"
    temp_col = teams_df[col].where(teams_df["is_active"], -np.inf)
    ranks = temp_col.groupby([teams_df["season_x"], teams_df["GW"]]).rank(
        method="min", ascending=False
    )
    teams_df[rank_col_name] = ranks.where(teams_df["is_active"])
    rank_cols.append(rank_col_name)

# Rank "bad" stats: lower is better (losses, goals conceded)
for col in ["cumulative_opponent_score", "cumulative_loss", "cumulative_draw"]:
    rank_col_name = f"{col}_rank"
    temp_col = teams_df[col].where(teams_df["is_active"], np.inf)
    ranks = temp_col.groupby([teams_df["season_x"], teams_df["GW"]]).rank(
        method="min", ascending=True
    )
    teams_df[rank_col_name] = ranks.where(teams_df["is_active"])
    rank_cols.append(rank_col_name)

# Drop helper column
teams_df = teams_df.drop(columns=["is_active"])

# Step F: Calculate Rolling Averages (Form over last last_number_gws GWs)
rolling_avg_cols = []
cols_for_rolling = ["team_score", "opponent_score", "win", "loss", "draw"]
for col in cols_for_rolling:
    avg_col_name = f"avg_{col}_last_{last_number_gws}"
    teams_df[avg_col_name] = teams_df.groupby(["season_x", "team_x"])[col].transform(
        lambda x: x.shift(1).rolling(window=last_number_gws, min_periods=1).mean()
    )
    rolling_avg_cols.append(avg_col_name)

# --- 3. Final Cleanup and Display ---
teams_df = teams_df.rename(
    columns={
        "cumulative_team_score": "cumulative_goals_scored",
        "cumulative_opponent_score": "cumulative_goals_conceded",
        "cumulative_win": "cumulative_wins",
        "cumulative_loss": "cumulative_losses",
        "cumulative_draw": "cumulative_draws",
    }
)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

# Ensure ranks don’t exceed 20, and fill rolling averages
teams_df[rank_cols] = teams_df[rank_cols].clip(upper=20)
teams_df[rolling_avg_cols] = teams_df[rolling_avg_cols].fillna(0.0)

print(teams_df.shape)
teams_df.head()

In [ ]:
teams_df.columns

In [ ]:
teams_df.info()

## Columns Handling


#### Column definitions


In [ ]:
# group by player and season, sort by gw, then get cumulative sum of set columns
cumulative_columns = [
    "assists",
    "bonus",
    "bps",
    "clean_sheets",
    "goals_conceded",
    "goals_scored",
    "minutes",
    "own_goals",
    "penalties_missed",
    "penalties_saved",
    "red_cards",
    "saves",
    "total_points",
    "yellow_cards",
]

# "creativity", "ict_index", "influence", "threat", # DK to put
minimum_90_minutes = 90  # To prevent anomalous values
per_90_columns = [
    "assists",
    "bonus",
    "bps",
    "clean_sheets",
    "goals_conceded",
    "goals_scored",
    # "own_goals",  # Dont think matters
    # "penalties_missed",  # Dont think matters
    # "penalties_saved",  # Dont think matters
    # "red_cards",  # Dont think matters
    "saves",
    "total_points",
    # "yellow_cards",  # Dont think matters
]

rolling_columns = [
    "assists",
    "bonus",
    "bps",
    "clean_sheets",
    "goals_conceded",
    "goals_scored",
    # "own_goals", # Dont think matters
    # "penalties_missed", # Dont think matters
    # "penalties_saved", # Dont think matters
    # "red_cards", # Dont think matters
    "saves",
    "total_points",
    # "yellow_cards",  # Dont think matters
]

In [ ]:
def add_columns(
    df,
    cumulative_cols,
    rolling_cols,
    per_90_columns,
    last_number_gws,
    minimum_90_minutes,
):
    # Make a copy of the original DataFrame to avoid modifying it directly
    df_temp = df.copy()

    # Sort the data by season, player (element), and gameweek to ensure proper cumulative/rolling order
    df_temp = df_temp.sort_values(["season_x", "element", "GW"])

    # --- 1️⃣ Cumulative feature computation ---
    # Compute cumulative sums for all columns in `cumulative_cols` within each player-season group
    df_temp[[f"{c}_cum" for c in cumulative_cols]] = df_temp.groupby(
        ["season_x", "element"], group_keys=False
    )[cumulative_cols].cumsum()

    # --- 2️⃣ Rolling (last N gameweeks) feature computation ---
    # For each player-season group, calculate rolling sums over the last `last_number_gws` gameweeks
    df_temp[[f"{c}_last_{last_number_gws}" for c in rolling_cols]] = (
        df_temp.groupby(["season_x", "element"], group_keys=False)[rolling_cols]
        # Apply a rolling window of `last_number_gws` size with at least 1 valid entry
        .rolling(window=last_number_gws, min_periods=1)
        # Sum values within each rolling window
        .sum()
        # Drop the extra multi-index created by groupby + rolling
        .reset_index(level=[0, 1], drop=True)
    )

    # --- 3️⃣ Per-90 stats with a capped denominator ---
    # For selected columns, compute per-90-minute values, capping low minute totals to avoid inflation
    for c in per_90_columns:
        df_temp[f"{c}_cum_per90"] = (
            # Divide cumulative value by cumulative minutes (clipped to at least `minimum_90_minutes`)
            df_temp[f"{c}_cum"]
            / df_temp["minutes_cum"].clip(lower=minimum_90_minutes)
            # Scale to a per-90-minute rate
            * 90
        )

    # Return the transformed DataFrame with cumulative, rolling, and per-90 features
    return df_temp

In [ ]:
df_addcolumns = add_columns(
    df,
    cumulative_columns,
    rolling_columns,
    per_90_columns,
    last_number_gws,
    minimum_90_minutes,
)

In [ ]:
df_addcolumns[
    [
        "total_points",
        "total_points_cum",
        "total_points_cum_per90",
        "minutes",
        "total_points_last_4",
    ]
].iloc[0:50]

In [ ]:
df_addcolumns.columns

In [ ]:
df = df_addcolumns.copy()
print(df.shape)
df.head()

## Shift columns


In [ ]:
columns_to_shift = ["team_x", "opp_team_name", "kickoff_time", "was_home", "GW"]

In [ ]:
def shift_columns(df, columns_to_shift):
    df_temp = df.copy()
    df_temp = df_temp.sort_values(["season_x", "element", "GW"])

    df_temp[[f"{col}_next" for col in columns_to_shift]] = df_temp.groupby(
        ["season_x", "element"], group_keys=False
    )[columns_to_shift].shift(-1)  # shift up by 1 row

    return df_temp


df_shiftcolumns = shift_columns(df, columns_to_shift)

In [ ]:
df = df_shiftcolumns.copy()
print(df.shape)
df.head()

## Finalize columns


In [ ]:
df = pd.get_dummies(df, columns=["position"], prefix="position")

In [ ]:
columns_needed = [
    "position_GK",
    "position_DEF",
    "position_MID",
    "position_FWD",
    # Unknown values
    "creativity",
    "ict_index",
    "influence",
    "threat",
    "value",
    "form",
    # Cumulative
    "assists_cum",
    "bonus_cum",
    "bps_cum",
    "clean_sheets_cum",
    "goals_conceded_cum",
    "goals_scored_cum",
    "minutes_cum",
    "own_goals_cum",
    "penalties_missed_cum",
    "penalties_saved_cum",
    "red_cards_cum",
    "saves_cum",
    "total_points_cum",
    "yellow_cards_cum",
    # Last x
    f"assists_last_{last_number_gws}",
    f"bonus_last_{last_number_gws}",
    f"bps_last_{last_number_gws}",
    f"clean_sheets_last_{last_number_gws}",
    f"goals_conceded_last_{last_number_gws}",
    f"goals_scored_last_{last_number_gws}",
    f"saves_last_{last_number_gws}",
    f"total_points_last_{last_number_gws}",
    # Per 90
    "assists_cum_per90",
    "bonus_cum_per90",
    "bps_cum_per90",
    "clean_sheets_cum_per90",
    "goals_conceded_cum_per90",
    "goals_scored_cum_per90",
    "saves_cum_per90",
    "total_points_cum_per90",
    # Use to get team values
    "team_x_next",
    "opp_team_name_next",
    "kickoff_time_next",
    "kickoff_time",
    "season_x",
    "was_home_next",
    "GW_next",  # ??
    "GW",
    # y predict
    "upcoming_total_points",
]

In [ ]:
# Removes rows where 'GW_next' is NaN (rows that are at GW38 therefore no GW39 to predict)
df = df.dropna(subset=["GW_next"])

In [ ]:
df = df[columns_needed]
print(df.shape)
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

## Joining df to teams_df


### Rename Team Columns


Done to make sure difference between player and team stats


In [ ]:
teams_df.columns

In [ ]:
new_team_column_names = {
    col: f"team_{col}"
    for col in teams_df.columns
    if col not in ["season_x", "team_x", "GW", "kickoff_time"]
}
new_team_column_names

In [ ]:
df.shape

In [ ]:
joined_df = df.merge(
    teams_df,
    how="left",
    left_on=["season_x", "kickoff_time", "team_x_next"],
    right_on=["season_x", "kickoff_time", "team_x"],
    suffixes=("", "_team"),
)

# Drop redundant columns (avoid duplicate GW/team_x columns)
joined_df = joined_df.drop(
    columns=["GW_team", "team_x_team", "GW_opp", "team_x_opp"], errors="ignore"
)

joined_df = joined_df.rename(columns=new_team_column_names)

# Check result
print(joined_df.shape)
joined_df.head()

In [ ]:
joined_df.columns

In [ ]:
joined_df.info()

In [ ]:
df = joined_df.copy()
print(df.shape)
df.head()

In [ ]:
# label encode was_home_next
df["was_home_next"] = (
    df["was_home_next"].map({"True": 1, "False": 0, True: 1, False: 0}).fillna(0)
)
# Convert all boolean columns to 0/1
bool_cols = df.select_dtypes(include=["bool"]).columns
df[bool_cols] = df[bool_cols].astype(int)
print(df.shape)
df.head()

# Encapsulating Preprocessing


In [ ]:
# Dropping unnecessary columns
drop_cols = [
    "team_x_next",
    "opp_team_name_next",
    "kickoff_time_next",
    "kickoff_time",
    "kickoff_time_opp",
    "season_x",
    "team_x",
    "GW",
    "GW_next",
    "kickoff_time_opp",
]

df = df.drop(columns=drop_cols, errors="ignore")
print(df.shape)

In [ ]:
df = df.dropna()
print(df.shape)
df.head()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

last_number_gws = 4
minimum_gws = 2  # Can change according to performance

# Define all position updates in one place
position_updates = [
    {
        "names": ["Jeffrey Schlupp", "Jairo Riedewald", "Declan Rice"],
        "seasons": ["2016-17", "2017-18"],
        "position": "DEF",
    },
    {
        "names": ["James Milner"],
        "seasons": ["2017-18"],
        "position": "DEF",
    },
    {
        "names": [
            "Eric Dier",
            "Fernando Luiz Rosa",
            "Ashley Young",
            "Daniel Amartey",
            "Robert Kenedy Nunes do Nascimento",
            "Ainsley Maitland-Niles",
            "Aaron Wan-Bissaka",
            "Oleksandr Zinchenko",
            "Anthony Martial",
            "Michail Antonio",
            "Richarlison de Andrade",
        ],
        "seasons": ["2016-17", "2017-18"],
        "position": "MID",
    },
    {
        "names": ["Roberto Firmino", "Jay Rodriguez", "Joshua King"],
        "seasons": ["2016-17"],
        "position": "MID",
    },
    {
        "names": ["Marcus Rashford", "Ayoze Pérez", "Pierre-Emerick Aubameyang"],
        "seasons": ["2016-17", "2017-18"],
        "position": "FWD",
    },
]

# ------- Column Definitions-------
cumulative_columns = [
    "assists",
    "bonus",
    "bps",
    "clean_sheets",
    "goals_conceded",
    "goals_scored",
    "minutes",
    "own_goals",
    "penalties_missed",
    "penalties_saved",
    "red_cards",
    "saves",
    "total_points",
    "yellow_cards",
]

# "creativity", "ict_index", "influence", "threat", # DK to put
minimum_90_minutes = 90  # To prevent anomalous values
per_90_columns = [
    "assists",
    "bonus",
    "bps",
    "clean_sheets",
    "goals_conceded",
    "goals_scored",
    # "own_goals",  # Dont think matters
    # "penalties_missed",  # Dont think matters
    # "penalties_saved",  # Dont think matters
    # "red_cards",  # Dont think matters
    "saves",
    "total_points",
    # "yellow_cards",  # Dont think matters
]

rolling_columns = [
    "assists",
    "bonus",
    "bps",
    "clean_sheets",
    "goals_conceded",
    "goals_scored",
    # "own_goals", # Dont think matters
    # "penalties_missed", # Dont think matters
    # "penalties_saved", # Dont think matters
    # "red_cards", # Dont think matters
    "saves",
    "total_points",
    # "yellow_cards",  # Dont think matters
]


columns_needed = [
    "position_GK",
    "position_DEF",
    "position_MID",
    "position_FWD",
    # Unknown values
    "creativity",
    "ict_index",
    "influence",
    "threat",
    "value",
    "form",
    # Cumulative
    "assists_cum",
    "bonus_cum",
    "bps_cum",
    "clean_sheets_cum",
    "goals_conceded_cum",
    "goals_scored_cum",
    "minutes_cum",
    "own_goals_cum",
    "penalties_missed_cum",
    "penalties_saved_cum",
    "red_cards_cum",
    "saves_cum",
    "total_points_cum",
    "yellow_cards_cum",
    # Last x
    f"assists_last_{last_number_gws}",
    f"bonus_last_{last_number_gws}",
    f"bps_last_{last_number_gws}",
    f"clean_sheets_last_{last_number_gws}",
    f"goals_conceded_last_{last_number_gws}",
    f"goals_scored_last_{last_number_gws}",
    f"saves_last_{last_number_gws}",
    f"total_points_last_{last_number_gws}",
    # Per 90
    "assists_cum_per90",
    "bonus_cum_per90",
    "bps_cum_per90",
    "clean_sheets_cum_per90",
    "goals_conceded_cum_per90",
    "goals_scored_cum_per90",
    "saves_cum_per90",
    "total_points_cum_per90",
    # Use to get team values
    "team_x_next",
    "opp_team_name_next",
    "kickoff_time_next",
    "kickoff_time",
    "season_x",
    "was_home_next",
    "GW_next",  # ??
    "GW",
    # y predict
    "upcoming_total_points",
]

drop_cols = [
    "team_x_next",
    "opp_team_name_next",
    "kickoff_time_next",
    "kickoff_time",
    "kickoff_time_opp",
    "season_x",
    "team_x",
    "GW",
    "GW_next",
    "kickoff_time_opp",
]

top_features = [
    "form",
    "bps_last_4",
    "total_points_last_4",
    "ict_index",
    "influence",
    "total_points_cum",
    "upcoming_total_points",
    "bps_cum",
    "bps_cum_per90",
    "minutes_cum",
    "clean_sheets_last_4",
    "goals_conceded_last_4",
    "clean_sheets_cum",
    "total_points_cum_per90",
    "bonus_cum",
    "creativity",
    "clean_sheets_cum_per90",
    "threat",
    "value",
    "goals_conceded_cum",
    "bonus_last_4",
]

target_col = "upcoming_total_points"

best_params = {
    "n_layers": 4,
    "n_units_l0": 510,
    "n_units_l1": 241,
    "n_units_l2": 222,
    "dropout_rate": 0.281975287168578,
    "lr": 0.00019165928146882533,
}


def teams_in_group(g):
    return set(g["opp_team_name"].dropna().astype(str).unique())


def populate_team_x(df):
    # Populating team_x for missing rows
    df_copy = df.copy()

    # force keys to be strings to make sure datatypes are compatible when joining
    for c in ["season_x", "GW", "fixture"]:
        df_copy[c] = df_copy[c].astype(str)

    fixture_teams = (
        df_copy.groupby(["season_x", "GW", "fixture"])
        # FIX: Added include_groups=False to silence the DeprecationWarning
        .apply(teams_in_group, include_groups=False)
        .reset_index(name="teams")
    )

    # Keep only fixtures with exactly 2 distinct teams
    fixture_teams = fixture_teams[fixture_teams["teams"].apply(lambda s: len(s) == 2)]

    fixture_teams[["team_a", "team_b"]] = fixture_teams["teams"].apply(
        lambda s: pd.Series(sorted(list(s)))
    )

    df_merged = df_copy.merge(
        fixture_teams[["season_x", "GW", "fixture", "team_a", "team_b"]],
        on=["season_x", "GW", "fixture"],
        how="left",
    )

    # mask to manage only rows who have team_x as null
    mask_null_teamx = df_merged["team_x"].isna() & df_merged["team_a"].notna()

    # FIX: Initialize the column with a string-compatible dtype ('object')
    # to silence the FutureWarning when assigning team names (strings) later.
    df_merged["team_x_inferred"] = np.nan
    df_merged["team_x_inferred"] = df_merged["team_x_inferred"].astype("object")

    # where opponent == team_a -> set to team_b
    mask_a = mask_null_teamx & (
        df_merged["opp_team_name"].astype(str) == df_merged["team_a"]
    )
    df_merged.loc[mask_a, "team_x_inferred"] = df_merged.loc[mask_a, "team_b"]

    # where opponent == team_b -> set to team_a
    mask_b = mask_null_teamx & (
        df_merged["opp_team_name"].astype(str) == df_merged["team_b"]
    )
    df_merged.loc[mask_b, "team_x_inferred"] = df_merged.loc[mask_b, "team_a"]

    df_merged["team_x_filled"] = df_merged["team_x"].fillna(
        df_merged["team_x_inferred"]
    )

    df_result = df.copy()
    # replace nulls in original team_x with filled values
    df_result["team_x"] = df_result["team_x"].fillna(df_merged["team_x_filled"])

    return df


def apply_position_updates(df, updates):
    df_temp = df.copy()
    for update in updates:
        condition = df["name"].isin(update["names"]) & df["season_x"].isin(
            update["seasons"]
        )
        df_temp.loc[condition, "position"] = update["position"]
    return df_temp


def add_team_scores(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df["team_score"] = np.where(df["was_home"], df["team_h_score"], df["team_a_score"])
    df["opponent_score"] = np.where(
        df["was_home"], df["team_a_score"], df["team_h_score"]
    )

    return df


def calculate_form(group):
    # Aggregate per GW (combine DGWs)
    gw_points = group.groupby("GW", as_index=False)["total_points"].sum()

    # Compute rolling sum of last 4 GWs (excluding current GW)
    gw_points["form"] = (
        gw_points["total_points"]
        # .shift(1)  # exclude current GW
        .rolling(window=4, min_periods=1)  # use up to 4 previous GWs
        .sum()
        .fillna(0)
        / 10  # scale like FPL (divided by 10)
    )

    # Merge back so both DGW rows share same form
    return group.merge(gw_points[["GW", "form"]], on="GW", how="left")


def get_teams_data(df):
    # Step A: Isolate team-match data and remove duplicates, include kickoff_time
    team_match_data = (
        df[["season_x", "team_x", "GW", "kickoff_time", "team_score", "opponent_score"]]
        .drop_duplicates()
        .copy()
    )

    # Step B: Sort values to ensure correct cumulative/rolling calculation order
    team_match_data = team_match_data.sort_values(
        by=["season_x", "team_x", "kickoff_time", "GW"]
    )

    # Step C: Determine win, loss, or draw for each individual match
    conditions = [
        team_match_data["team_score"] > team_match_data["opponent_score"],
        team_match_data["team_score"] < team_match_data["opponent_score"],
        team_match_data["team_score"] == team_match_data["opponent_score"],
    ]
    team_match_data["win"] = np.select(conditions, [1, 0, 0], default=0)
    team_match_data["loss"] = np.select(conditions, [0, 1, 0], default=0)
    team_match_data["draw"] = np.select(conditions, [0, 0, 1], default=0)

    # --- Create the final DataFrame, starting with the base data ---
    teams_df = team_match_data.copy()

    # Step D: Calculate Cumulative Stats
    cols_to_sum = ["team_score", "opponent_score", "win", "loss", "draw"]
    cumulative_cols = [f"cumulative_{col}" for col in cols_to_sum]
    teams_df[cumulative_cols] = teams_df.groupby(["season_x", "team_x"])[
        cols_to_sum
    ].cumsum()

    # --- NEW STEP D.5: Identify which teams are active in each season ---
    active_teams_index = (
        df.dropna(subset=["team_score", "opponent_score"])
        .set_index(["season_x", "team_x"])
        .index.unique()
    )
    teams_df["is_active"] = teams_df.set_index(["season_x", "team_x"]).index.isin(
        active_teams_index
    )

    # --- MODIFIED STEP E: Calculate Ranks for Active Teams Only ---
    rank_cols = []

    # Rank "good" stats: higher is better (wins, goals scored)
    for col in ["cumulative_team_score", "cumulative_win"]:
        rank_col_name = f"{col}_rank"
        temp_col = teams_df[col].where(teams_df["is_active"], -np.inf)
        ranks = temp_col.groupby([teams_df["season_x"], teams_df["GW"]]).rank(
            method="min", ascending=False
        )
        teams_df[rank_col_name] = ranks.where(teams_df["is_active"])
        rank_cols.append(rank_col_name)

    # Rank "bad" stats: lower is better (losses, goals conceded)
    for col in ["cumulative_opponent_score", "cumulative_loss", "cumulative_draw"]:
        rank_col_name = f"{col}_rank"
        temp_col = teams_df[col].where(teams_df["is_active"], np.inf)
        ranks = temp_col.groupby([teams_df["season_x"], teams_df["GW"]]).rank(
            method="min", ascending=True
        )
        teams_df[rank_col_name] = ranks.where(teams_df["is_active"])
        rank_cols.append(rank_col_name)

    # Drop helper column
    teams_df = teams_df.drop(columns=["is_active"])

    # Step F: Calculate Rolling Averages (Form over last last_number_gws GWs)
    rolling_avg_cols = []
    cols_for_rolling = ["team_score", "opponent_score", "win", "loss", "draw"]
    for col in cols_for_rolling:
        avg_col_name = f"avg_{col}_last_{last_number_gws}"
        teams_df[avg_col_name] = teams_df.groupby(["season_x", "team_x"])[
            col
        ].transform(
            lambda x: x.shift(1).rolling(window=last_number_gws, min_periods=1).mean()
        )
        rolling_avg_cols.append(avg_col_name)

    # --- 3. Final Cleanup and Display ---
    teams_df = teams_df.rename(
        columns={
            "cumulative_team_score": "cumulative_goals_scored",
            "cumulative_opponent_score": "cumulative_goals_conceded",
            "cumulative_win": "cumulative_wins",
            "cumulative_loss": "cumulative_losses",
            "cumulative_draw": "cumulative_draws",
        }
    )

    # Ensure ranks don’t exceed 20, and fill rolling averages
    teams_df[rank_cols] = teams_df[rank_cols].clip(upper=20)
    teams_df[rolling_avg_cols] = teams_df[rolling_avg_cols].fillna(0.0)

    return teams_df


def add_columns(
    df,
    cumulative_cols,
    rolling_cols,
    per_90_columns,
    last_number_gws,
    minimum_90_minutes,
):
    # Make a copy of the original DataFrame to avoid modifying it directly
    df_temp = df.copy()

    # Sort the data by season, player (element), and gameweek to ensure proper cumulative/rolling order
    df_temp = df_temp.sort_values(["season_x", "element", "GW"])

    # --- 1️⃣ Cumulative feature computation ---
    # Compute cumulative sums for all columns in `cumulative_cols` within each player-season group
    df_temp[[f"{c}_cum" for c in cumulative_cols]] = df_temp.groupby(
        ["season_x", "element"], group_keys=False
    )[cumulative_cols].cumsum()

    # --- 2️⃣ Rolling (last N gameweeks) feature computation ---
    # For each player-season group, calculate rolling sums over the last `last_number_gws` gameweeks
    df_temp[[f"{c}_last_{last_number_gws}" for c in rolling_cols]] = (
        df_temp.groupby(["season_x", "element"], group_keys=False)[rolling_cols]
        # Apply a rolling window of `last_number_gws` size with at least 1 valid entry
        .rolling(window=last_number_gws, min_periods=1)
        # Sum values within each rolling window
        .sum()
        # Drop the extra multi-index created by groupby + rolling
        .reset_index(level=[0, 1], drop=True)
    )

    # --- 3️⃣ Per-90 stats with a capped denominator ---
    # For selected columns, compute per-90-minute values, capping low minute totals to avoid inflation
    for c in per_90_columns:
        df_temp[f"{c}_cum_per90"] = (
            # Divide cumulative value by cumulative minutes (clipped to at least `minimum_90_minutes`)
            df_temp[f"{c}_cum"]
            / df_temp["minutes_cum"].clip(lower=minimum_90_minutes)
            # Scale to a per-90-minute rate
            * 90
        )

    # Return the transformed DataFrame with cumulative, rolling, and per-90 features
    return df_temp


def shift_columns(df, columns_to_shift):
    df_temp = df.copy()
    df_temp = df_temp.sort_values(["season_x", "element", "GW"])

    df_temp[[f"{col}_next" for col in columns_to_shift]] = df_temp.groupby(
        ["season_x", "element"], group_keys=False
    )[columns_to_shift].shift(-1)  # shift up by 1 row

    return df_temp


def prepare_data(
    path: Path = Path(path_global),
    split_scale: bool = True,
    top_features=top_features,
    is_drop_columns=False,
) -> pd.DataFrame:
    df = pd.read_csv(path)

    # ------ Data Cleaning ------
    # Impute missing team_x values
    df = populate_team_x(df)

    # Convert GKP to GK for consistency
    df["position"] = df["position"].replace({"GKP": "GK"})

    # Handle Minimum Number of GWS
    df = df[
        df.groupby(["season_x", "element"])["element"].transform("size") >= minimum_gws
    ]

    df_positions = apply_position_updates(df, position_updates)
    df = df_positions.copy()

    df.sort_values(by=["season_x", "element", "kickoff_time"], inplace=True)
    df["upcoming_total_points"] = df.groupby(["season_x", "element"])[
        "total_points"
    ].shift(-1)
    mask_gw_38 = df["GW"] == 38
    df.loc[mask_gw_38, "upcoming_total_points"] = df.loc[mask_gw_38, "total_points"]

    df = add_team_scores(df)

    form_df = df.copy()

    form_df = form_df.sort_values(
        by=["season_x", "element", "GW", "kickoff_time"]
    ).reset_index(drop=True)

    # Apply to each player (element) per season
    form_df = form_df.groupby(["season_x", "element"], group_keys=False).apply(
        calculate_form
    )

    # Optional: fill NaN (shouldn’t appear except for first GW)
    form_df["form"] = form_df["form"].fillna(0)

    df = form_df.copy()

    teams_df = get_teams_data(df)

    df_addcolumns = add_columns(
        df,
        cumulative_columns,
        rolling_columns,
        per_90_columns,
        last_number_gws,
        minimum_90_minutes,
    )

    df = df_addcolumns.copy()

    df_shiftcolumns = shift_columns(df, columns_to_shift)
    df = df_shiftcolumns.copy()

    # One-Hot Encoding for position
    df = pd.get_dummies(df, columns=["position"], prefix="position")

    # Removes rows where 'GW_next' is NaN (rows that are at GW38 therefore no GW39 to predict)
    df = df.dropna(subset=["GW_next"])

    # Merge with team stats
    joined_df = df.merge(
        teams_df,
        how="left",
        left_on=["season_x", "kickoff_time", "team_x_next"],
        right_on=["season_x", "kickoff_time", "team_x"],
        suffixes=("", "_team"),
    )

    # Drop redundant columns (avoid duplicate GW/team_x columns)
    joined_df = joined_df.drop(
        columns=["GW_team", "team_x_team", "GW_opp", "team_x_opp"], errors="ignore"
    )

    # Rename team columns to make it different than player stats
    new_team_column_names = {
        col: f"team_{col}"
        for col in teams_df.columns
        if col not in ["season_x", "team_x", "GW", "kickoff_time"]
    }
    joined_df = joined_df.rename(columns=new_team_column_names)

    df = joined_df.copy()

    # label encode was_home_next
    df["was_home_next"] = (
        df["was_home_next"].map({"True": 1, "False": 0, True: 1, False: 0}).fillna(0)
    )

    # Convert all boolean columns to 0/1
    bool_cols = df.select_dtypes(include=["bool"]).columns
    df[bool_cols] = df[bool_cols].astype(int)

    if is_drop_columns:
        df = df.drop(columns=drop_cols, errors="ignore")

    df = df.dropna()

    if not split_scale:
        if top_features is not None:
            return df[top_features]
        else:
            return df

    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Standardize (important for some methods)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

    corr = df.corr()[target_col].sort_values(ascending=False)

    # 3. UNIVARIATE FEATURE SELECTION (F-test)
    selector = SelectKBest(score_func=f_regression, k=20)
    selector.fit(X_scaled, y)
    scores = pd.Series(selector.scores_, index=X.columns).sort_values(ascending=False)

    # 4. CONSOLIDATE RESULTS
    results = pd.DataFrame(
        {
            "Correlation": corr,
            "F_test": scores,
        }
    ).fillna(0)

    # Normalize scores to compare
    results = results.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    results["Combined_Score"] = results.mean(axis=1)
    results = results.sort_values("Combined_Score", ascending=False)

    # 5. SELECT TOP FEATURES AND CREATE NEW DF
    top_features = results.head(21).index.tolist()  # select top 20
    df_selected = df[top_features]

    df = df_selected.copy()

    return df

In [ ]:
df_compare = prepare_data(path=path_global, split_scale=False)

In [ ]:
for column in df_compare.columns:
    mask = df_compare[column] != df[column]
    different_rows = df.loc[
        mask
    ]  # or df_compare.loc[mask], depending on which you want to see
    if len(different_rows) > 0:
        print(different_rows)

# Feature Selection


In [ ]:
df.info()

In [ ]:
n_features = 21  # (20 features + y predict)

`n_features` was decided upon iterative function -> check `auto_feature_selection_fn.txt`


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# ==========================================
# 1. DATA PREPARATION
# ==========================================
target_col = "upcoming_total_points"

X = df.drop(columns=[target_col])
y = df[target_col]

# Standardize (important for some methods)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# ==========================================
# 2. CORRELATION ANALYSIS
# ==========================================
corr = df.corr()[target_col].sort_values(ascending=False)
plt.figure(figsize=(6, 12))
sns.barplot(x=corr.values[:20], y=corr.index[:20], palette="viridis")
plt.title("Top 20 Features by Pearson Correlation with Target")
plt.xlabel("Correlation Coefficient")
plt.tight_layout()
plt.show()

print("\nTop correlated features:")
print(corr.head(10))

# ==========================================
# 3. UNIVARIATE FEATURE SELECTION (F-test)
# ==========================================
selector = SelectKBest(score_func=f_regression, k=20)
selector.fit(X_scaled, y)
scores = pd.Series(selector.scores_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(6, 12))
sns.barplot(x=scores.values[:20], y=scores.index[:20], palette="magma")
plt.title("Top 20 Features by F-test (Linear Relationship)")
plt.xlabel("F-score")
plt.tight_layout()
plt.show()

print("\nTop F-test features:")
print(scores.head(10))

# ==========================================
# 4. CONSOLIDATE RESULTS
# ==========================================
results = pd.DataFrame(
    {
        "Correlation": corr,
        "F_test": scores,
    }
).fillna(0)

# Normalize scores to compare
results = results.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
results["Combined_Score"] = results.mean(axis=1)
results = results.sort_values("Combined_Score", ascending=False)

plt.figure(figsize=(7, 12))
sns.barplot(x=results["Combined_Score"][:20], y=results.index[:20], palette="plasma")
plt.title("Top 20 Features (Combined Importance Score)")
plt.xlabel("Normalized Importance")
plt.tight_layout()
plt.show()

print("\n===== Top Features (Combined Score) =====")
print(results.head(n_features))

# ==========================================
# 5. SELECT TOP FEATURES AND CREATE NEW DF
# ==========================================
top_features = results.head(n_features).index.tolist()  # select top 20
df_selected = df[top_features]

print(f"\n✅ df_selected created with {len(top_features)} features + target column.")
print(f"Shape: {df_selected.shape}")
print("Selected features:")
print(top_features)
df_selected.head()

In [ ]:
df = df_selected.copy()
print(df.shape)
df.head()

In [ ]:
df.info()

# Full EDA after preprocessing


In [ ]:
full_eda(df)

# Training the FFNN Model


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# separate features & target
y = df["upcoming_total_points"].values
X_temp = df.drop(columns=["upcoming_total_points"])
X_columns = X_temp.columns
X = X_temp.values

In [ ]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import pickle
import os

# Define a file path for your scaler
scaler_filename = "model/ffnn_feature_scaler.pkl"

# Save the trained scaler
with open(scaler_filename, "wb") as file:
    pickle.dump(scaler, file)

print(f"Scaler saved successfully to {scaler_filename}")

In [ ]:
# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
print("X_train_t shape:", X_train_t.shape)
print("y_train_t shape:", y_train_t.shape)
print("X_test_t shape:", X_test_t.shape)
print("y_test_t shape:", y_test_t.shape)

In [ ]:
# Create Dataloaders
train_data = TensorDataset(X_train_t, y_train_t)
test_data = TensorDataset(X_test_t, y_test_t)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
class FFNN(nn.Module):
    def __init__(
        self,
        input_dim,
        n_layers=3,
        n_units_l0=218,
        n_units_l1=233,
        n_units_l2=70,
        dropout_rate=0.10236066118288575,
    ):
        super(FFNN, self).__init__()

        layers = []
        in_dim = input_dim
        hidden_units = [n_units_l0, n_units_l1, n_units_l2][:n_layers]

        for h in hidden_units:
            layers.append(nn.Linear(in_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            in_dim = h

        layers.append(nn.Linear(in_dim, 1))  # output layer
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

We used Optuna library for HPO, it takes long time to run so it shouldn't be run every time. Here are the `best_trial` outputs:<br><br>
Tuned HPs: {'n_layers': 4, 'n_units_l0': 510, 'n_units_l1': 241, 'n_units_l2': 222, 'n_units_l3': 296, 'dropout_rate': 0.281975287168578, 'lr': 0.00019165928146882533, 'batch_size': 32} <br>

Check `optuna_train_cell.txt` for more details.


In [ ]:
# Initialize model, loss, optimizer, batch_size
best_params = {
    "n_layers": 4,
    "n_units_l0": 510,
    "n_units_l1": 241,
    "n_units_l2": 222,
    "dropout_rate": 0.281975287168578,
    "lr": 0.00019165928146882533,
}

model = FFNN(
    input_dim=X_train.shape[1],
    n_layers=best_params["n_layers"],
    n_units_l0=best_params["n_units_l0"],
    n_units_l1=best_params["n_units_l1"],
    n_units_l2=best_params["n_units_l2"],
    dropout_rate=best_params["dropout_rate"],
)

optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()

In [ ]:
epochs = 30
train_losses = []
val_losses = []

best_val_loss = float("inf")
patience = 5  # how many epochs to wait after no improvement
tolerance = 0.001  # minimum improvement in val_loss to count as progress
no_improve_count = 0
best_model_state = None

for epoch in range(epochs):
    # ---- Training ----
    model.train()
    total_train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # ---- Validation ----
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            preds = model(xb)
            loss = criterion(preds, yb)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)
    val_losses.append(avg_val_loss)

    print(
        f"Epoch [{epoch + 1}/{epochs}] | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Loss: {avg_val_loss:.4f}"
    )

    # ---- Early Stopping Logic ----
    if best_val_loss - avg_val_loss > tolerance:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()  # save best weights
        no_improve_count = 0
    else:
        no_improve_count += 1
        if no_improve_count >= patience:
            print(
                f"\nEarly stopping at epoch {epoch + 1} "
                f"(no improvement in val loss for {patience} epochs)."
            )
            break

# ---- Load best model ----
if best_model_state is not None:
    model.load_state_dict(best_model_state)

In [ ]:
# Save the trained model
model_filename = "model/ffnn_fpl_model.pth"
torch.save(model, model_filename)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(
    range(1, len(train_losses) + 1), train_losses, marker="o", label="Training Loss"
)
plt.plot(range(1, len(val_losses) + 1), val_losses, marker="s", label="Validation Loss")
plt.title("Training & Validation Loss with Early Stopping")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate
model.eval()
with torch.no_grad():
    preds = model(X_test_t)
    preds_np = preds.cpu().numpy()
    y_test_np = y_test_t.cpu().numpy()

    # Compute metrics
    mse = mean_squared_error(y_test_np, preds_np)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_np, preds_np)
    r2 = r2_score(y_test_np, preds_np)

    # Print all
    print("\nModel Evaluation Metrics:")
    print(f"MAE  (Mean Absolute Error): {mae:.4f}")
    print(f"MSE  (Mean Squared Error):  {mse:.4f}")
    print(f"RMSE (Root MSE):            {rmse:.4f}")
    print(f"R²   (R-squared):           {r2:.4f}")


# Baseline Model (LR)


In [ ]:
from sklearn.linear_model import LinearRegression

# ==========================================
# BASELINE MODEL - LINEAR REGRESSION
# ==========================================
baseline_model = LinearRegression()

# Train
baseline_model.fit(X_train, y_train)

# Predict
y_pred_baseline = baseline_model.predict(X_test)

# Compute metrics
mse_b = mean_squared_error(y_test, y_pred_baseline)
rmse_b = np.sqrt(mse_b)
mae_b = mean_absolute_error(y_test, y_pred_baseline)
r2_b = r2_score(y_test, y_pred_baseline)

print("\n===== Baseline Model Evaluation =====")
print(f"MAE  (Mean Absolute Error): {mae_b:.4f}")
print(f"MSE  (Mean Squared Error):  {mse_b:.4f}")
print(f"RMSE (Root MSE):            {rmse_b:.4f}")
print(f"R²   (R-squared):           {r2_b:.4f}")

# XAI


In [ ]:
X_train_np = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
X_test_np = X_test.values if isinstance(X_test, pd.DataFrame) else X_test

X_train_np = X_train_np.astype(np.float32)
X_test_np = X_test_np.astype(np.float32)

### SHAP


In [ ]:
import shap
import torch

shap.initjs()


# Define a wrapper to convert numpy -> torch -> model output
def ffnn_predict(x_numpy):
    with torch.no_grad():
        x_tensor = torch.tensor(x_numpy, dtype=torch.float32)
        preds = model(x_tensor).numpy()
    return preds.flatten()  # <--- flatten keeps it 1D, safe for SHAP


# Create SHAP explainer using KernelExplainer (model-agnostic)
background = X_train_np[np.random.choice(X_train_np.shape[0], 100, replace=False)]
explainer = shap.KernelExplainer(ffnn_predict, background)

print("X_train_np:", X_train_np.shape)
print("X_test_np:", X_test_np.shape)
print("background:", background.shape)

with torch.no_grad():
    sample_pred = model(torch.tensor(X_test_np[:1], dtype=torch.float32))
print("Model output shape:", sample_pred.shape)

# Explain a subset of test samples
shap_values = explainer.shap_values(X_test_np[:50], nsamples=100)

# Plot summary of global feature importance
if isinstance(X_temp, pd.DataFrame):
    feature_names = X_temp.columns.tolist()
else:
    feature_names = [f"feature_{i}" for i in range(X_test_np.shape[1])]

shap.summary_plot(shap_values, X_test_np[:50], feature_names=feature_names)

In [ ]:
# If shap_values is a list (of one element), unwrap it
if isinstance(shap_values, list):
    shap_values = shap_values[0]  # unwrap first element

import shap

shap.initjs()  # enables interactive HTML plots

i = 0
shap.force_plot(
    explainer.expected_value,
    shap_values[i, :],
    X_test_np[i, :],
    feature_names=feature_names,
    matplotlib=False,  # important for interactive HTML
)

In [ ]:
shap.plots.bar(
    shap.Explanation(
        values=shap_values[i, :],
        base_values=explainer.expected_value,
        data=X_test_np[i, :],
        feature_names=feature_names,
    )
)

### LIME


In [ ]:
from lime import lime_tabular
import IPython.display
import lime.explanation

from IPython.display import display, HTML

# Initialize LIME explainer
lime_explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_train_np, feature_names=feature_names, mode="regression"
)

# Explain one test instance
i = 5  # index of test sample to explain
exp = lime_explainer.explain_instance(X_test_np[i], ffnn_predict, num_features=10)


html_exp = exp.as_html()
display(HTML(html_exp))

# Inference Function


In [ ]:
df_final = prepare_data(
    path=Path(path_global),
    split_scale=False,
    top_features=None,
    is_drop_columns=False,
)

In [ ]:
df_final[df_final["name"] == "Kevin De Bruyne"][
    [
        "GW",
        "upcoming_total_points",
        "total_points",
        "total_points_cum",
        "total_points_cum_per90",
        "opp_team_name",
        "opp_team_name_next",
    ]
].iloc[4:26]

In [ ]:
df_final[df_final["name"] == "Mohamed Salah"][
    [
        "GW",
        "upcoming_total_points",
        "total_points",
        "total_points_cum",
        "total_points_cum_per90",
        "opp_team_name",
        "opp_team_name_next",
        "season_x",
    ]
].iloc[-26:]

In [ ]:
def run_pipeline(
    raw_data_path: Path = Path(path_global),
    model_path: Path = Path("model/ffnn_fpl_model.pth"),
    scaler_path: Path = Path("model/ffnn_feature_scaler.pkl"),
    gw_to_predict: int | None = None,
    name: str | None = None,
    season_x: str | None = None,
    target_col: str = "upcoming_total_points",
    top_features: list = top_features,
):
    # 1. Prepare data
    df_raw = prepare_data(path=raw_data_path, split_scale=False, top_features=None)

    if gw_to_predict is not None:
        raw_rows = df_raw[
            (df_raw["GW_next"] == gw_to_predict)
            & (df_raw["name"] == name)
            & (df_raw["season_x"] == season_x)
        ]
    else:
        raw_rows = df_raw

    df_prepared = df_raw[top_features + [target_col]]

    # Select specific row if index provided
    if gw_to_predict is not None:
        df_prepared = df_prepared[
            (df_raw["GW_next"] == gw_to_predict)
            & (df_raw["name"] == name)
            & (df_raw["season_x"] == season_x)
        ]

    # 2. Load scaler
    with open(scaler_path, "rb") as file:
        scaler = pickle.load(file)

    # 3. Scale features
    X = df_prepared.drop(columns=[target_col])
    X = scaler.transform(X)

    model = torch.load(model_path, weights_only=False)

    model.eval()
    with torch.no_grad():
        x_tensor = torch.tensor(X, dtype=torch.float32)
        preds = model(x_tensor)
        preds_np = preds.cpu().numpy()

    for i, pred in enumerate(preds_np):
        raw_row = raw_rows.iloc[i]
        print(
            f"Predicted upcoming_total_points for GW {raw_row['GW_next']} for player {raw_row['name']} vs {raw_row['opp_team_name_next']}: {pred[0]:.2f}, while the actual points are {raw_row['upcoming_total_points']}, with an error of {abs(pred[0] - raw_row['upcoming_total_points']):.2f}"
        )

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    run_pipeline(
        raw_data_path=Path(path_global),
        model_path=Path("model/ffnn_fpl_model.pth"),
        scaler_path=Path("model/ffnn_feature_scaler.pkl"),
        gw_to_predict=8,  # 24 for a DGW example
        season_x="2020-21",
        name="Kevin De Bruyne",
        target_col="upcoming_total_points",
        top_features=top_features,
    )

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    run_pipeline(
        raw_data_path=Path(path_global),
        model_path=Path("model/ffnn_fpl_model.pth"),
        scaler_path=Path("model/ffnn_feature_scaler.pkl"),
        gw_to_predict=24,  # 24 for a DGW example
        season_x="2022-23",
        name="Mohamed Salah",
        target_col="upcoming_total_points",
        top_features=top_features,
    )